In [ ]:
# ============================
# STEP 1: Install dependencies
# ============================
!pip install kaggle pycocotools opencv-python albumentations matplotlib -q

import os, random
import cv2
import numpy as np
import matplotlib.pyplot as plt
from pycocotools.coco import COCO
import albumentations as A

# ============================
# STEP 2: Upload kaggle.json
# ============================
from google.colab import files
files.upload()   # Upload kaggle.json (from Kaggle Account -> API -> Create New API Token)

!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

# ============================
# STEP 3: Download COCO 2017 dataset
# ============================
!kaggle datasets download -d awsaf49/coco-2017-dataset
!unzip -q coco-2017-dataset.zip -d coco

# ============================
# STEP 4: Paths
# ============================
imgDir  = "coco/val2017"
annFile = "coco/annotations/instances_val2017.json"

# Load COCO
coco = COCO(annFile)

# ============================
# STEP 5: Get category ID for "backpack"
# ============================
backpack_catId = coco.getCatIds(catNms=["backpack"])[0]

# Get only images containing backpacks
backpack_imgIds = coco.getImgIds(catIds=[backpack_catId])
random.shuffle(backpack_imgIds)

# Select 5 sample backpack images
sample_imgIds = backpack_imgIds[:5]

# ============================
# STEP 6: Augmentation pipeline
# ============================
transform = A.Compose([
    A.Resize(255, 255),
    A.RandomRotate90(),
    A.HorizontalFlip(p=0.5),
    A.VerticalFlip(p=0.5),
    A.ShiftScaleRotate(shift_limit=0.2, scale_limit=0.3, rotate_limit=60, p=0.7),
    A.RandomBrightnessContrast(p=0.5),
    A.HueSaturationValue(p=0.5),
    A.GaussianBlur(blur_limit=(3, 7), p=0.4),
    A.RandomGamma(p=0.4),
    A.CoarseDropout(max_holes=8, max_height=40, max_width=40, p=0.3)
])

# ============================
# STEP 7: Process backpack images
# ============================
for idx, imgId in enumerate(sample_imgIds, 1):
    imgInfo = coco.loadImgs(imgId)[0]

    # Load image
    imgPath = os.path.join(imgDir, imgInfo['file_name'])
    image = cv2.cvtColor(cv2.imread(imgPath), cv2.COLOR_BGR2RGB)

    # Load backpack annotations
    annIds = coco.getAnnIds(imgIds=imgInfo['id'], catIds=[backpack_catId])
    anns = coco.loadAnns(annIds)

    # Build binary mask (backpack=1, background=0)
    mask = np.zeros((imgInfo['height'], imgInfo['width']), dtype=np.uint8)
    for ann in anns:
        mask = np.maximum(mask, coco.annToMask(ann).astype(np.uint8))

    # Resize + normalize image
    image_resized = cv2.resize(image, (255, 255)).astype(np.float32) / 255.0
    mask_resized = cv2.resize(mask, (255, 255), interpolation=cv2.INTER_NEAREST)

    # Binary mask (backpack=white, background=black)
    binary_mask = (mask_resized * 255).astype(np.uint8)

    # ============================
    # Apply augmentation
    # ============================
    augmented = transform(image=image, mask=mask)
    aug_img, aug_mask = augmented["image"], augmented["mask"]

    aug_img_norm = aug_img.astype(np.float32) / 255.0
    aug_binary_mask = (aug_mask * 255).astype(np.uint8)

    # ============================
    # Show results
    # ============================
    fig, ax = plt.subplots(1, 3, figsize=(15, 5))
    ax[0].imshow(image_resized)
    ax[0].set_title("Original Backpack (Resized + Normalized)")
    ax[0].axis("off")

    ax[1].imshow(aug_img_norm)
    ax[1].set_title("Augmented Backpack Image")
    ax[1].axis("off")

    ax[2].imshow(aug_binary_mask, cmap="gray")
    ax[2].set_title("Binary Mask (Backpack=White, BG=Black)")
    ax[2].axis("off")

    plt.show()
